In [1]:
import numpy as np
from itertools import combinations, product
from myf import calculate_f_jk, calculate_g_x
from joblib import Parallel, delayed
import os
import pickle
from tslearn.barycenters import softdtw_barycenter

In [3]:
def precompute_distances(
    profiles,
    n_jobs=-1,
):
    n = len(profiles)

    # initial the matrix

    dtw_matrix = np.full((n, n), np.inf)

    indices = list(combinations(range(n), 2))

    # Calculate the score between 2 signals

    distances = Parallel(n_jobs=n_jobs)(
        delayed(calculate_f_jk)(profiles[i], profiles[j]) for i, j in indices
    )

    # Assign score into matrix

    for (i, j), dist in zip(indices, distances):

        dtw_matrix[i, j] = dist

    masked_arr = np.ma.masked_where(np.isinf(dtw_matrix), dtw_matrix)

    # np.save("masked_arr_data.npy", masked_arr.data)

    np.savez(
        "masked_arr_both_multi_157.npz", data=masked_arr.data, mask=masked_arr.mask
    )

    return masked_arr

In [ ]:
if __name__ == "__main__":

    train_files = os.listdir("smoothed_multi_train")
    train_list = []
    for train_file in train_files:
        file_path = os.path.join("smoothed_multi_train", train_file)
        train_list.append(np.load(file_path))
    train_data = np.array(train_list)

    # first get masked_arr and save
    masked_arr = precompute_distances(train_data)
    